In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
path="D:\Python\Elo"
def prepare_data():
    path="D:\Python\Elo"
    a=pd.read_csv(os.path.join(path,'historical_transactions.csv'))
    a['category_1'] = a['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    a['category_2'].fillna(1.0,inplace=True)
    a['category_3'].fillna('A',inplace=True)
    a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    a['installments'].replace(-1, np.nan,inplace=True)
    a['installments'].replace(999, np.nan,inplace=True)
    a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    a['price'] = a['purchase_amount'] / a['installments']
    a['purchase_date'] = pd.to_datetime(a['purchase_date'])
    a['weekend'] = (a.purchase_date.dt.weekday >=5).astype(np.int32)
    aggs={
            'purchase_amount':['mean','sum','count','std'],
        'price':['mean','sum','count','std'],
        'merchant_category_id':['nunique'],
        'category_1':['sum', 'mean'],
        'category_2':['nunique'],
        'category_3':['nunique'],
        'city_id': ['nunique'],
       'subsector_id':['nunique'],
        'authorized_flag':['mean'],
        'weekend':['mean'],
        'card_id':['size'],
        'installments':['mean','sum']}
    a=a.pivot_table(index='card_id',columns='month_lag',aggfunc=aggs)
    col=['hist_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
    a.columns=col
    data=pd.read_csv(os.path.join(path,'train.csv'))
    data=data[['card_id','target']].merge(a,how='left',on='card_id')
    data=data.fillna(0)
    y=data['target'].values
    for i in range(2,16):
        if i==2:
            x=data.values[:,i::14]
        elif i==3:
            x=np.stack((x,data.values[:,i::14]),axis=1)
        else:
            x=np.append(x,data.values[:,i::14].reshape((x.shape[0],1,x.shape[2])),axis=1)
    a=pd.read_csv(os.path.join(path,'new_merchant_transactions.csv'))
    a['category_1'] = a['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    a['category_2'].fillna(1.0,inplace=True)
    a['category_3'].fillna('A',inplace=True)
    a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    a['installments'].replace(-1, np.nan,inplace=True)
    a['installments'].replace(999, np.nan,inplace=True)
    a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    a['price'] = a['purchase_amount'] / a['installments']
    a['purchase_date'] = pd.to_datetime(a['purchase_date'])
    a['weekend'] = (a.purchase_date.dt.weekday >=5).astype(np.int32)
    aggs={
            'purchase_amount':['mean','sum','count','std'],
        'price':['mean','sum','count','std'],
        'merchant_category_id':['nunique'],
        'category_1':['sum', 'mean'],
        'category_2':['nunique'],
        'category_3':['nunique'],
        'city_id': ['nunique'],
       'subsector_id':['nunique'],
        'authorized_flag':['mean'],
        'weekend':['mean'],
        'card_id':['size'],
        'installments':['mean','sum']}
    a=a.pivot_table(index='card_id',columns='month_lag',aggfunc=aggs)
    col=['hist_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
    a.columns=col
    data=pd.read_csv(os.path.join(path,'train.csv'))
    data=data[['card_id','target']].merge(a,how='left',on='card_id')
    data=data.fillna(0)
    for i in range(2,4):
        x=np.append(x,data.values[:,i::2].reshape((x.shape[0],1,x.shape[2])),axis=1)
    del data;a;gc.collect() 
    return x,y

In [4]:
X,y=prepare_data()

In [6]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,Conv2D, Bidirectional,GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D,BatchNormalization,GRU,LSTM
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.engine.topology import Layer
from sklearn.metrics import f1_score
from keras import backend as K
from keras.initializers import *
from keras.layers import concatenate
from sklearn import metrics
import tensorflow as tf
data=pd.read_csv(os.path.join(path,'train.csv'))
data['outliers'] = 0
data.loc[data['target'] < -30, 'outliers'] = 1
data=data[['card_id','outliers']]
skf=StratifiedKFold(5,shuffle=True,random_state=100)
val_pr=np.zeros(len(y))
for train_index,val_index in skf.split(data['card_id'],data['outliers'].values):
    train_x=X[train_index]
    train_y=y[train_index]
    val_x=X[val_index]
    val_y=y[val_index]
    inp=Input(shape=(X.shape[1],X.shape[2]))
 #   x=Embedding(x.shape[0],x.shape[2], trainable=False)
    x=Bidirectional(LSTM(90,return_sequences=True))(inp)
    x=Bidirectional(GRU(90,return_sequences=True))(x)
    avg_pool=GlobalAveragePooling1D()(x)
    max_pool=GlobalMaxPooling1D()(x)
    conc=concatenate([avg_pool,max_pool])
    conc=Dense(64,activation='relu')(conc)
    conc=Dropout(0.1)(conc)
    outp=Dense(1,activation='linear')(conc)
    model=Model(inputs=inp,outputs=outp)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
    model.fit(train_x,train_y, batch_size=512,epochs=5,validation_data=(val_x,val_y))
    val_pr[val_index]=np.squeeze(model.predict(val_x, batch_size=1024, verbose=1))

Train on 161533 samples, validate on 40384 samples
Epoch 1/5
161533/161533 [==============================] - 165s 1ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/5
161533/161533 [==============================] - 159s 982us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/5
161533/161533 [==============================] - 168s 1ms/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/5
161533/161533 [==============================] - 136s 845us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/5
40384/40384 [==============================] - 10s 260us/step


ValueError: shape mismatch: value array of shape (40384,1) could not be broadcast to indexing result of shape (40384,)

In [8]:
train_y

array([-0.8202826 ,  0.39291325,  0.68805599, ..., -2.74082069,
       -4.67658938, -1.85941301])

In [100]:
a=pd.read_csv(os.path.join(path,'new_merchant_transactions.csv'))
a['category_1'] = a['category_1'].map({'Y': 1, 'N': 0}).astype(int)
a['category_2'].fillna(1.0,inplace=True)
a['category_3'].fillna('A',inplace=True)
a['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
a['installments'].replace(-1, np.nan,inplace=True)
a['installments'].replace(999, np.nan,inplace=True)
a['authorized_flag'] = a['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
a['price'] = a['purchase_amount'] / a['installments']
a['purchase_date'] = pd.to_datetime(a['purchase_date'])
a['weekend'] = (a.purchase_date.dt.weekday >=5).astype(np.int32)
aggs={
        'purchase_amount':['mean','sum','count','std'],
    'price':['mean','sum','count','std'],
    'merchant_category_id':['nunique'],
    'category_1':['sum', 'mean'],
    'category_2':['nunique'],
    'category_3':['nunique'],
    'city_id': ['nunique'],
   'subsector_id':['nunique'],
    'authorized_flag':['mean'],
    'weekend':['mean'],
    'card_id':['size'],
    'installments':['mean','sum']}
a=a.pivot_table(index='card_id',columns='month_lag',aggfunc=aggs)
col=['hist_'+s1+'_'+s2+'_'+str(s3) for (s1,s2,s3) in a.columns.tolist()]
a.columns=col
data=pd.read_csv(os.path.join(path,'train.csv'))
data=data[['card_id','target']].merge(a,how='left',on='card_id')

In [104]:
data.values[:,2::2].shape

(201917, 20)

In [105]:
for i in range(2,4):
    x=np.append(x,data.values[:,i::2].reshape((x.shape[0],1,x.shape[2])),axis=1)

In [18]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,Conv2D, Bidirectional,GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D,BatchNormalization,GRU,LSTM
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.engine.topology import Layer
from sklearn.metrics import f1_score
from keras import backend as K
from keras.initializers import *
from keras.layers import concatenate
from sklearn import metrics
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
path="D:\Python\Elo"
df_train=pd.read_csv(os.path.join(path,'result.csv'))
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train_columns=['card_id','outliers','target',
                 'hist_month_lag_mean', 'hist_month_diff_mean',
       'hist_fathers_day_2017_mean', 'hist_duration_min',
       'hist_purchase_date_diff', 'hist_purchase_date_uptonow',
       'hist_last_installments_sum_0', 'hist_purchase_amount_mean_-4',
       'new_purchase_mean_last_3month_mean']
df_train=df_train[df_train_columns]
df_train_columns=['hist_month_lag_mean', 'hist_month_diff_mean',
       'hist_fathers_day_2017_mean', 'hist_duration_min',
       'hist_purchase_date_diff', 'hist_purchase_date_uptonow',
       'hist_last_installments_sum_0', 'hist_purchase_amount_mean_-4',
       'new_purchase_mean_last_3month_mean']
scal=StandardScaler()
df_train[df_train_columns]=df_train[df_train_columns].replace([np.inf, -np.inf], np.nan)
df_train[df_train_columns]=df_train[df_train_columns].fillna(0)
df_train[df_train_columns]=scal.fit_transform(df_train[df_train_columns])
skf=StratifiedKFold(5,shuffle=True,random_state=100)
val_pr=np.zeros(len(df_train['target']))
for train_index,val_index in skf.split(df_train['card_id'],df_train['outliers'].values):
    train_x=df_train[df_train_columns].loc[train_index]
    train_y=df_train['target'].loc[train_index]
    val_x=df_train[df_train_columns].loc[val_index]
    val_y=df_train['target'].loc[val_index]
    inp=Input(shape=(len(df_train_columns),))
 #   x=Embedding(x.shape[0],x.shape[2], trainable=False)
    x=Dense(1024,activation='relu')(inp)
    x=Dropout(0.25)(x)
    x=BatchNormalization()(x)
    x=Dense(512,activation='relu')(x)
    x=BatchNormalization()(x)
    x=Dropout(0.25)(x)
    x=Dense(256,activation='relu')(x)
    x=BatchNormalization()(x)
    x=Dropout(0.25)(x)
    x=Dense(128,activation='relu')(x)
    x=BatchNormalization()(x)
    x=Dropout(0.25)(x)
    outp=Dense(1,activation='linear')(x)
    model=Model(inputs=inp,outputs=outp)
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
    model.fit(train_x,train_y, batch_size=512,epochs=50,validation_data=(val_x,val_y))
    val_pr[val_index]=np.squeeze(model.predict(val_x, batch_size=1024, verbose=1))

Train on 161533 samples, validate on 40384 samples
Epoch 1/50
161533/161533 [==============================] - 32s 200us/step - loss: 14.7326 - mean_squared_error: 14.7326 - val_loss: 14.3998 - val_mean_squared_error: 14.3998
Epoch 2/50
161533/161533 [==============================] - 27s 167us/step - loss: 14.3177 - mean_squared_error: 14.3177 - val_loss: 14.2288 - val_mean_squared_error: 14.2288
Epoch 3/50
161533/161533 [==============================] - 22s 136us/step - loss: 14.1864 - mean_squared_error: 14.1864 - val_loss: 14.2144 - val_mean_squared_error: 14.2144
Epoch 4/50
161533/161533 [==============================] - 27s 169us/step - loss: 14.1295 - mean_squared_error: 14.1295 - val_loss: 14.2060 - val_mean_squared_error: 14.2060
Epoch 5/50
161533/161533 [==============================] - 27s 166us/step - loss: 14.1041 - mean_squared_error: 14.1041 - val_loss: 14.2214 - val_mean_squared_error: 14.2214
Epoch 6/50
161533/161533 [==============================] - 23s 140us/step

161533/161533 [==============================] - 26s 160us/step - loss: 13.8370 - mean_squared_error: 13.8370 - val_loss: 14.0541 - val_mean_squared_error: 14.0541
Epoch 48/50
161533/161533 [==============================] - 27s 167us/step - loss: 13.8340 - mean_squared_error: 13.8340 - val_loss: 14.0654 - val_mean_squared_error: 14.0654
Epoch 49/50
161533/161533 [==============================] - 27s 169us/step - loss: 13.8373 - mean_squared_error: 13.8373 - val_loss: 14.1233 - val_mean_squared_error: 14.1233
Epoch 50/50
40384/40384 [==============================] - 2s 46us/step
Train on 161533 samples, validate on 40384 samples
Epoch 1/50
161533/161533 [==============================] - 30s 187us/step - loss: 14.7896 - mean_squared_error: 14.7896 - val_loss: 14.2201 - val_mean_squared_error: 14.2201
Epoch 2/50
161533/161533 [==============================] - 23s 141us/step - loss: 14.3107 - mean_squared_error: 14.3107 - val_loss: 14.2345 - val_mean_squared_error: 14.2345
Epoch 3/50


161533/161533 [==============================] - 23s 143us/step - loss: 13.8614 - mean_squared_error: 13.8614 - val_loss: 13.9995 - val_mean_squared_error: 13.9995
Epoch 44/50
161533/161533 [==============================] - 24s 147us/step - loss: 13.8734 - mean_squared_error: 13.8734 - val_loss: 13.9827 - val_mean_squared_error: 13.9827
Epoch 45/50
161533/161533 [==============================] - 23s 144us/step - loss: 13.8890 - mean_squared_error: 13.8890 - val_loss: 14.0009 - val_mean_squared_error: 14.0009
Epoch 46/50
161533/161533 [==============================] - 22s 137us/step - loss: 13.8355 - mean_squared_error: 13.8355 - val_loss: 14.0043 - val_mean_squared_error: 14.0043
Epoch 47/50
161533/161533 [==============================] - 22s 138us/step - loss: 13.8329 - mean_squared_error: 13.8329 - val_loss: 14.0432 - val_mean_squared_error: 14.0432
Epoch 48/50
161533/161533 [==============================] - 22s 136us/step - loss: 13.8494 - mean_squared_error: 13.8494 - val_loss

KeyboardInterrupt: 

In [17]:
np.sqrt(mean_squared_error(val_pr,df_train['target']))

3.745708434892737